# Mars Seismic Data Visualization


This is a notebook to download and visualize Mars seismic event data from the xxxx 3-component seismiometer

Here is a great readme on downloading the data and formatting it from NASA: https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/readme.txt

First need to:
 -  Choose the events by downloading the catalog, remove events that have default lat/long (4.5024, 135.6234)
 -  Download the raw data automatically from NASA (only horizontal component currently (BHU)
 -  Aggregate the data and compute differentiation
 -  Plot the events in panels

In [8]:
import pandas as pd
import numpy as np
import os
import urllib.request
import matplotlib.pyplot as plt
import requests
from io import StringIO

# place to save data
basedir = "C:/home/data/mars/"


def getEvents(starttime=None,endtime=None):
    url = "https://service.iris.edu/irisws/mars-event/1/query?minmagnitude=0&format=text"
    response = requests.get(url)
    plain_text = response.text
    lines = plain_text.strip().split('\n')
    data = [line.split('|') for line in lines]
    df = pd.DataFrame(data[1:], columns=data[0])
    df = df[(df['Latitude']!="4.5024") & (df['Latitude']!="135.6234")]
    df['Sol'] = df['ContributorID'].str[1:4]
    df['Time'] = pd.to_datetime(df['Time'])
    df['Year'] = df['Time'].dt.year.astype(str)
    df['DOY'] = (df['Time'].dt.dayofyear).astype(str).str.zfill(3)
    if starttime:
        df = df[df['Time']>starttime]
    if endtime:
        df = df[df['Time']<endtime]
    return df

events = getEvents()

# Download the seismicity data test (need come up with a list for event format)
# url_base = "https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/continuous_waveform/elyse/2021/122/"
# file_name = "xb.elyse.02.bhu.2021.122.7.a.csv"
# url = url_base+file_name

# Time the Insight lander landed on 
lander_time = pd.to_datetime("2018-11-26T11:52:59.0Z")

def download_raw_data(url,file_name,save = False):
    d = pd.read_csv(url,skiprows=19, delimiter=',')
    # Option to save data
    if save:
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        d.to_csv(basedir+file_name,index=False)
    return d

def download_all_events(allevents):
    for indx,event in allevents.iterrows():
        doy = event['DOY']
        year = event['Year']
        url_base = f"https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/continuous_waveform/elyse/{year}/{doy}/"
        for ix in range(15):
            try:
                # Only horizontal component being grabbed
                file_name = f"xb.elyse.02.bhu.{year}.{doy}.{ix}.a.csv"
                url = url_base+file_name
                download_raw_data(url,file_name,save=True)
                print(f'downloaded {file_name}')
                break
            except:
                continue

def read_downloaded_events(levents=None):
    dlist = []
    for file in os.listdir(basedir):
        sub = pd.read_csv(basedir+file).drop(columns=['Unnamed: 0'],errors='ignore').rename(columns={" Sample":"Sample"})
        sub['name'] = os.path.splitext(os.path.basename(file))[0]
        print('reading:',os.path.splitext(os.path.basename(file))[0])
        dlist.append(sub)
    return dlist
                

# Uncomment to download all events to disk in base_dir
#download_all_events(events)

dlist = read_downloaded_events()

data = pd.concat(dlist)

print('done')

reading... xb.elyse.02.bhu.2021.122.7.a
reading... xb.elyse.02.bhu.2022.002.2.a
reading... xb.elyse.02.bhu.2022.034.7.a
reading... xb.elyse.02.bhu.2022.124.3.a
done


In [9]:
def process_data(data):
    data['Time'] = pd.to_datetime(data['Time'])
    data['Sols'] = (data['Time'] - lander_time) / np.timedelta64(1, 'D')
    data['Amplitude_Normalized'] = (data['Sample']-min(data['Sample']))/(max(data['Sample'])-min(data['Sample']))
    data['Amplitude_Diff'] = data['Sample'].diff()
    return data

pdata = process_data(data)
pdata

,Time,Sample,name,Sols,Amplitude_Normalized,Amplitude_Diff
0,2021-05-02 00:00:00.016000+00:00,-4178,xb.elyse.02.bhu.2021.122.7.a,887.504873,0.492981,NaN
1,2021-05-02 00:00:00.066000+00:00,-4206,xb.elyse.02.bhu.2021.122.7.a,887.504873,0.492973,-28.0
2,2021-05-02 00:00:00.116000+00:00,-4202,xb.elyse.02.bhu.2021.122.7.a,887.504874,0.492974,4.0
3,2021-05-02 00:00:00.166000+00:00,-4207,xb.elyse.02.bhu.2021.122.7.a,887.504875,0.492972,-5.0
4,2021-05-02 00:00:00.216000+00:00,-4272,xb.elyse.02.bhu.2021.122.7.a,887.504875,0.492952,-65.0
...,...,...,...,...,...,...
1727993,2022-05-04 23:59:59.662000+00:00,-4776,xb.elyse.02.bhu.2022.124.3.a,1255.504869,0.492792,426.0
1727994,2022-05-04 23:59:59.712000+00:00,-4563,xb.elyse.02.bhu.2022.124.3.a,1255.504869,0.492859,213.0
1727995,2022-05-04 23:59:59.762000+00:00,-3126,xb.elyse.02.bhu.2022.124.3.a,1255.504870,0.493315,1437.0
1727996,2022-05-04 23:59:59.812000+00:00,-878,xb.elyse.02.bhu.2022.124.3.a,1255.504871,0.494029,2248.0


In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go
import plotly

# Set column names
yax = "Amplitude_Diff"
xax = "Time"

# Get the unique event names
event_names = data['name'].unique()

# Create subplots with a shared x-axis
fig = sp.make_subplots(rows=len(event_names), cols=1, shared_xaxes=True, subplot_titles=event_names, vertical_spacing=0.05)

# Loop through event names and add a trace for each event
for idx, event_name in enumerate(event_names):
    event_data = data[data['name'] == event_name]
    
    trace = go.Scatter(
        x=event_data[xax],
        y=event_data[yax],
        mode="lines",
        name=event_name,
        line=dict(width=2),
    )
    
    fig.add_trace(trace, row=idx + 1, col=1)

# Update the layout with axis labels and a title
fig.update_layout(
    title="Seismic Data from InSight SEIS",
    xaxis=dict(title="Date Time"),
    yaxis=dict(title="Amplitude Normalized"),
)

# Display the interactive plot
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)